## Background remover + Object extraction

For the data set it is important to have as much reliable data as possible. As my model should only analyze the plushies themselves, it's beneficial to remove the background behind the plushies. For this, I build a background remover script using Python library rembg. <br>
Because some images also contain multiple plushies in one image, I also used a script to extract the those in multiple images. The script I used is from an answer from StackOverflow that uses OpenCV: https://stackoverflow.com/a/56604981

Install the necessary libraries for image processing and manipulation

In [22]:
pip install pillow
pip install rembg

SyntaxError: invalid syntax (1647248791.py, line 1)

Import the models

In [4]:
from PIL import Image
from rembg import remove 
from fastai.vision.all import *
import os
import cv2

D:\Jenny\Code\anaconda3\envs\example\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Declares an input folder where the original content is stored, and an output folder where the edited images will be
If the output folder does not exist, the script will create one

In [12]:
folder_input = r"C:\Users\jenny\Desktop\minor\notebooks\authentic bootleg plush identifier\test images"
folder_output = r"C:\Users\jenny\Desktop\minor\notebooks\authentic bootleg plush identifier\test images\removed background"

if not os.path.exists(folderoutput_path):
    os.makedirs(folderoutput_path)

The script goes through every file in an folder to find images with a '.png', '.jpg' or '.jpeg' extension. If it finds one, it declares the path of the input image.<br>
If the color model of the images are RGBA or P, the model converts it to RGB to output an image with transparency.<br>
The script then removed the background in the images and saves it in the output folder

In [13]:
for filename in os.listdir(folder_input):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        with Image.open(os.path.join(folder_input, filename)) as img:
            
            if img.mode in ("RGBA", "P"):
                img = img.convert("RGB")
                
            output = remove(img)
            output.save(os.path.join(folder_output, filename + ".png"))

When an image has multiple plushies in one image, object extraction can be done with Python scripting with OpenCV. Because the script doesn't support transparent images, they would have to be replaced with white pixels and then removed again after the extraction process <br>
Like the background remover script, input and output folders are specified. If the output folder does not exist, it will be made automatically

In [29]:
folder_input_t2w =  r"C:\Users\jenny\Desktop\minor\notebooks\authentic bootleg plush identifier\test images\removed background"
output_folder_t2w =  r"C:\Users\jenny\Desktop\minor\notebooks\authentic bootleg plush identifier\test images\transparant to white"

# Create the output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

The script goes through every file in an folder to find images with a '.png', '.jpg' or '.jpeg' extension. If it finds one, it declares the path of the input image. <br>
First a new image is created with a white background in the same size as the original image, which is 224 x 224 pixels. The image with the plush is then overlaid on the white background image and saved in the output folder

In [32]:
for filename in os.listdir(folder_input_t2w):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        with Image.open(os.path.join(folder_input_t2w, filename)) as image:
            
            canvas = Image.new('RGB', image.size, (255,255,255,255))
            canvas.paste(image, mask=image)
            canvas.save(os.path.join(output_folder_t2w, filename))

Declares an input folder where the images with white background is stored, and an output folder where the objects of the original images will be seperated. <br>
If the output folder does not exist, the script will create one

In [ ]:
input_folder = r"C:\Users\jenny\Desktop\minor\notebooks\authentic bootleg plush identifier\test images\transparant to white"
output_folder = r"C:\Users\jenny\Desktop\minor\notebooks\authentic bootleg plush identifier\test images\extract objects"

# Create the output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

The images go though several processes to obtain a binary image to extract the objects: <br>
- Greyscale (converting the image to grey monochrome)
- Gaussian blur (blurring an image to minimize image noise)
- Otsu’s threshold (converts the image in white and black based on the found foreground and background)
- Dilate (expands the white region representing the foreground)
<br>

When the binary image is created, the script then finds the contours of the objects. It then creates bounding boxes and extracts those, and saves the objects from the bounding boxes in seperate images.<br>

In [33]:
for image_name in os.listdir(input_folder):
    if image_name.endswith(".png") or image_name.endswith(".jpg") or image_name.endswith(".jpeg"):
        # Step 2: Object detection
        image = cv2.imread(os.path.join(input_folder, image_name))
        original = image.copy()
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
        dilate = cv2.dilate(thresh, kernel, iterations=1)

        # Find contours, obtain bounding box coordinates, and extract ROI
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        image_number = 0
        for c in cnts:
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(image, (x, y), (x + w, y + h), (36, 255, 12), 2)
            ROI = original[y:y + h, x:x + w]
            cv2.imwrite(f"{output_folder}/ROI_{image_name}_{image_number}.png", ROI)
            image_number += 1

FileNotFoundError: [WinError 3] Het systeem kan het opgegeven pad niet vinden: 'C:\\Users\\jenny\\Desktop\\minor\\notebooks\\authentic bootleg plush identifier\\test images\\ggg'